In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder

# Ансамбли моделей

Как мы уже увидели, деревья склонны переобучаться, что отрицательно сказывается на качестве конечной модели. Однако, слабые модели можно объединяться в ансамбли и выбирать среди них конечный ответ путём голосования. Так, комбинация простых моделей может выявлять более сложные зависимости. 

Возникает вопрос - как правильно комбинировать модели чтобы получать хорошие ансамбли? Хорошо зарекомендовавшим себя подходом является бэггинг. Своё название он берёт от принципа - **b**ootstrap **agg**regat**ing**. 

## Bootstrap

Метод бутстрэпа заключается в создании обучающих подвыборок, путём выбора с возвращением. То есть, из обучающей выборки $X$ размера $N$ выберем $N$ объектов, однако, перед выбором следующего объекта, вернём выбранный объект обратно в обучающую выборку. Это как брать из мешка наугад шарик, а потом снова возвращать его обратно. Из-за этого в подвыборке окажутся повторы и она будет отличаться от исходной.

Повторив процедуру $k$ раз, мы получим набор независимых подвыборок $X_1, X_2, ... , X_k$.

## Bagging

Имея в распоряжении набор независимых подвыборок $X_1, X_2, ... , X_k$, мы можем научить на каждой из них свой классификатор $a_1(X_1), a_2(X_2), ..., a_k(X_k)$. 

Причем необязательно это должны быть алгоритмы одного типа. Конечное же решение по классификации объекта будет принимать *итоговый классификатор*: 
$$a(x) = \frac{1}{k}\sum_{i=1}^{k}a_i(x)$$

## Random Forest

Разумеется, бэггинг применим и к решающим деревьям. Набор (лес) деревьев учат на подвыборках, полученных бутстрэпом. Такой подход даже имеет собственное название *Random Forest*. 

Но Random Forest обладает ещё одной интересной особенностью - при построении очередного предиката учитываются не все признаки, а только их случайная выборка. Это делается для того, чтобы избежать однородности получающихся деревьев.

Попробуем применить решающие деревья на практике и при помощи кросс-валидации сравнить результаты одного дерева, бэггинга и результаты работы случайного леса. В качестве обучающей выборки возьмём датасет titanic.

In [ ]:
titanic = sns.load_dataset('titanic')
titanic.head()

Описание колонок:

*  *survived* — поле в котором указано спасся человек (1) или нет (0)
*  *pclass* — содержит социально-экономический статус: 
  * высокий 
  * средний
  * низкий
*  *sex* — пол пассажира
*  *age* — возраст
*  *sibsp* — содержит информацию о количестве родственников 2-го порядка (муж, жена, братья, сестры)
*  *parch* — содержит информацию о количестве родственников на борту 1-го порядка (мать, отец, дети)
*  *fare* — цена билета
*  *Embarked* — порт посадки
    * C — Cherbourg
    * Q — Queenstown
    * S — Southampton
    
    
Дальше поля почему-то дублируют оставшиеся, кроме deck:
*  *dect* — литера палубы


Отбросим дублируемые колонки:

In [ ]:
titanic = sns.load_dataset('titanic')
titanic = titanic.drop(
    columns=['class', 'who', 'adult_male', 'embark_town', 'alive', 'alone'])

И поищем пропуски в данных:

In [ ]:
titanic.isna().sum() / titanic.shape[0]

И вот тут мы на распутье. Чем заполнять пропущенные значения? Если для палубы можно ввести дополнительную переменную, то как быть с возрастом? Начнём с палубы:

In [ ]:
titanic['deck'].unique()

Просто заполним пропуски литерой U (unknown):

In [ ]:
titanic['deck'] = titanic['deck'].astype('str')
titanic.loc[titanic['deck'].eq('nan'), 'deck'] = 'U'

Так же поступим с полем embarked:

In [ ]:
titanic['embarked'] = titanic['embarked'].astype('str')
titanic.loc[titanic['embarked'].eq('nan'), 'embarked'] = 'U'

А вот с возрастом всё не так просто. Очевидно, что возраст очень важная фича. И заполнять пропуски средним значением для 20% данных означет сильно запутать наш классификатор.

А давайте совсем избавимся от них:

In [ ]:
titanic = titanic.dropna(subset=['age'])

Для полного счастья осталось сделать энкодинг признаков, потому, что дерево не станет работать с категориальными переменными. Для этого нам нужно закодировать признаки. В этом нам поможет `LabelEncoder` из библиотеки `sklearn.preprocessing`.

In [ ]:
encoders = {}
for column in titanic.columns:
    if titanic[column].dtype == 'object':
        encoder = LabelEncoder()
        encoder.fit(titanic[column])
        titanic[column] = encoder.transform(titanic[column])
        encoders[column] = encoder

In [ ]:
def predict_by_tree(X_train, y_train, X_test, y_test):
    tree = DecisionTreeClassifier(criterion='entropy',
                                  random_state=42,
                                  max_depth=None)
    tree.fit(X_train, y_train)
    predict = tree.predict(X_test)

    return predict


def predict_by_RF(X_train, y_train, X_test, y_test):
    forest = RandomForestClassifier(n_estimators=100,
                                    criterion='entropy',
                                    random_state=42)
    forest.fit(X_train, y_train)
    predict = forest.predict(X_test)

    return predict


def accuracy(ans, pred):
    acc = np.abs(pred == ans).mean() * 100
    return acc


X = titanic.drop(columns=['survived'])
y = titanic['survived']

np.random.seed(42)
train_split = y.apply(lambda x: True if np.random.rand() < 0.8 else False)
test_split = ~train_split

X_train, X_test = X[train_split], X[test_split]
y_train, y_test = y[train_split], y[test_split]

tree_prediction = predict_by_tree(X_train, y_train, X_test, y_test)
rf_prediction = predict_by_RF(X_train, y_train, X_test, y_test)

print(f'Just one tree accuracy: {accuracy(y_test, tree_prediction):.2f}%')
print(f'Random Forest accuracy: {accuracy(y_test, rf_prediction):.2f}%')

*N.B. Попробуйте поварьировать глубину дерева и критерий, чтобы посмотреть как меняются результаты*

И, чтобы быть уверенным в результатах, прокатим через кросс-валидацию:

*N.B. Я надеюсь, что у вас выполнены ячейки выше и функции `predict_by_tree` и `predict_by_RF` определены*

In [ ]:
titanic = titanic.sample(frac=1, replace=False, random_state=42)
X = titanic.drop(columns=['survived'])
y = titanic['survived'].copy()

splits = np.linspace(0, y.size, 6)
splits = splits.astype(int)

tree_acc = []
rf_acc = []

for hi, low in zip(splits[1:], splits[:-1]):
    train_mask = np.ones_like(y.values, dtype=bool)
    train_mask[low:hi] = 0
    test_mask = ~train_mask

    X_train, X_test = X[train_mask], X[test_mask]
    y_train, y_test = y[train_mask], y[test_mask]

    tree_prediction = predict_by_tree(X_train, y_train, X_test, y_test)
    rf_prediction = predict_by_RF(X_train, y_train, X_test, y_test)

    tree_acc.append(accuracy(y_test, tree_prediction))
    rf_acc.append(accuracy(y_test, rf_prediction))

tree_mean_acc = np.mean(tree_acc)
rf_mean_acc = np.mean(rf_acc)

print(f'Just one tree cross-validation accuracy: {tree_mean_acc:.2f}%')
print(f'Random Forest cross-validation accuracy: {rf_mean_acc:.2f}%')

Выводы предлагается сделать самостоятельно. Тем более, они должны зависеть от гиперпараметров моделей.

# Приложение 1: Пара слов о метриках

Количество выживших отличается от количество спасённых, а это делает наш датасет *несбалансированным*. А несбалансированный датасет это всегда плохо. 
- В смысле обучения: практически все модели будут отдавать предпочтения классам, которые встречаются чаще
- В смысле оценки результатов: нужно быть осторожным с использованием простых метрик, вроде *accuracy*

Меньше абстракции, больше живых примеров:

In [ ]:
survived = np.sum(titanic['survived'] == 1)
drowned = np.sum(titanic['survived'] == 0)

print(f'Количество выживших: {survived}')
print(f'Количество утопленников (F): {drowned}')

А вот проблема с метриками. Даже если объявить всех пассажиров утопленниками, `accuracy` будет выглядеть не так уж и плохо:

In [ ]:
ans = titanic['survived']
pred = np.zeros_like(ans)

print(f'Cruel prediction accuracy: {accuracy(ans, pred):.2f}%')

А теперь давайте посмотрим на другие метрики и сравним результаты с приличными предсказаниями:

In [ ]:
X = titanic.drop(columns=['survived'])
y = titanic['survived']

np.random.seed(42)
train_split = y.apply(lambda x: True if np.random.rand() < 0.8 else False)
test_split = ~train_split

X_train, X_test = X[train_split], X[test_split]
y_train, y_test = y[train_split], y[test_split]

rf_prediction = predict_by_RF(X_train, y_train, X_test, y_test)
skynet_prediction = np.zeros_like(y_test)

Следующий пример показывает, что далеко не все метрики репрезентативны поодиночке. Например, [чувствительность и специфичность](https://en.wikipedia.org/wiki/Sensitivity_and_specificity) хороши, но поодиночке могут вводить в заблуждение:

In [ ]:
def sepecifity_score(ans, pred):
    TN, FP, FN, TP = confusion_matrix(ans, pred).ravel()
    return TN / (TN + FP)


def recall_score(ans, pred):
    TN, FP, FN, TP = confusion_matrix(ans, pred).ravel()
    return TP / (TP + FN)


sepecifity, skynet_sepecifity = sepecifity_score(
    y_test, rf_prediction), sepecifity_score(y_test, skynet_prediction)
recall, skynet_recall = recall_score(y_test, rf_prediction), recall_score(
    y_test, skynet_prediction)

print(f'Specifity: RF {sepecifity:.2f}  Skynet {skynet_sepecifity:.2f}')
print(f'Recall:    RF {recall:.2f}  Skynet {skynet_recall:.2f}')

Достаточно репрезентативной будет метрика `confusion matrix`. 

Это квадратная матрица со стороной, равной количеству классов в модели. Её строки соответствуют оценкам (то бишь, предсказаниям модели), а столбцы - правильным ответам. 

Элементы такой матрице соответствуют количеству семплов, которые были отнесены моделью в класс, равный номеру строки, и имеющие ответ, соответствующий столбцу.

В нашем случае у нас получается матрица 2х2, Negatives - невыжившие, Positives - выжившие

\begin{bmatrix}
    TN & FP \\
    FN & TP
\end{bmatrix}

- TN (true negatives): утонувшие люди, которые были классифицированы моделью как утонувшие
- FP (false positives): утонувшие люди, которые были классифицированы моделью как спасшиеся
- FN (false negatives): спасшиеся люди, которые были классифицированы моделью как утонувшие
- TP (true positives): спасшиеся люди, которые были классифицированы моделью как спасшиеся

In [ ]:
from sklearn.metrics import confusion_matrix

print('Памятка:')
print('TN FP\nFN TP\n')

TN, FP, FN, TP = confusion_matrix(y_test, rf_prediction).ravel()
print('Нормальная модель:')
print(f'{TN} {FP}\n{FN} {TP}\n')

TN, FP, FN, TP = confusion_matrix(y_test, skynet_prediction).ravel()
print('Модель с замашками скайнета:')
print(f'{TN} {FP}\n{FN} {TP}')

# Приложение 2: Важность признаков (коротенькое)

Random Forest, бустеры и другие модели, которые являются композициями алгоритмов позволяют оценить *важность* каждого из признаков для принятия решений. 

Поскольку Random Forest усекает часть признаков когда строит различные деревья, можно оценить влияние каждого признака на конечный результат смотря на ошибку каждой из моделей. 

Информация о важности признаков хранится в атрибуте Random Forest модели:

In [ ]:
X = titanic.drop(columns=['survived'])
y = titanic['survived']

np.random.seed(42)
train_split = y.apply(lambda x: True if np.random.rand() < 0.8 else False)
test_split = ~train_split

X_train, X_test = X[train_split], X[test_split]
y_train, y_test = y[train_split], y[test_split]

forest = RandomForestClassifier(n_estimators=100,
                                criterion='entropy',
                                random_state=42)
forest.fit(X_train, y_train)

print(forest.feature_importances_)

Как видите, важность признаков идёт без всяких подписей. Исправим это безобразие:

In [ ]:
importances = {
    feature: importance
    for feature, importance in zip(X_train.columns,
                                   forest.feature_importances_)
}

for feature in sorted(importances, key=importances.get, reverse=True):
    print(f'Importance of {feature}: {importances[feature]:.2f}')